Refs:
* https://web.maths.unsw.edu.au/~fkuo/sobol/joe-kuo-notes.pdf
* https://web.maths.unsw.edu.au/~fkuo/sobol/
* https://researchcommons.waikato.ac.nz/bitstream/handle/10289/967/Joe%20constructing.pdf?sequence=1&isAllowed=y
* http://www.pbr-book.org/3ed-2018/Sampling_and_Reconstruction/(0,_2)-Sequence_Sampler.html
* http://www.pbr-book.org/3ed-2018/Sampling_and_Reconstruction/Sobol_Sampler.html

In [ ]:
import numpy as np
from matplotlib import pyplot

In [ ]:
def ReverseBits(bits):
    bits = (bits << 16) | (bits >> 16)
    bits = ((bits & 0x00ff00ff) << 8) | ((bits & 0xff00ff00) >> 8)
    bits = ((bits & 0x0f0f0f0f) << 4) | ((bits & 0xf0f0f0f0) >> 4)
    bits = ((bits & 0x33333333) << 2) | ((bits & 0xcccccccc) >> 2)
    bits = ((bits & 0x55555555) << 1) | ((bits & 0xaaaaaaaa) >> 1)
    return bits

def To01(x):
    return x / float(1<<32)

In [ ]:
def PlotMatrix(ax, C):
    a = np.zeros((32,32), np.uint8)
    for i in range(32):
        for j in range(32):
            a[i,j] = (1) if C[j]&(1<<i) else 0
    ax.imshow(a)

In [ ]:
def MakeTheMatrix(s, a, m):
    # The generator matrix
    # s: Polynomial degree
    # a: Polynomial coefficients (binary).
    # m: Initial direction numbers
    C = np.zeros(32, dtype=np.uint32)
    for k in range(s):
        C[k] = m[k]
        #print (C[k], bin(C[k]))
    for k in range(s, 32):
        # Generate m[k]
        pow2 = 1<<s
        C[k] ^= C[k-s]
        C[k] ^= pow2*C[k-s]
        for i in range(s-1):
            pow2 = pow2>>1
            #print (f"C[{k}] ... i{i}, pow2 {pow2}, a {a&1}, C[{k-s+i+1}]")
            if (a & (1<<(s-2-i))):
                C[k] ^= pow2*C[k-s+i+1]
            assert(k-s+i+1 >= 0)
            assert(pow2>=2)
        #print (C[k], bin(C[k]))
    # Turn the m's into the v's
    for k in range(0, 32):
        # The shift aligns the bits to the left.
        C[k] = C[k]<<(32-k-1)
    return C

In [ ]:
def MakeIdentityGenerator():
    a = np.zeros(32, np.uint32)
    for i in range(32):
        a[i] = 1<<(31-i)
    return a

In [ ]:
def MultGenerator(C, x):
    out = np.uint32(0)
    i = 0;
    while x != 0:
        if x & 1:
            out ^= C[i]
        i += 1
        x = x>>1
    return out

In [ ]:
def Generate(N, C):
    for i in range(N):
        yield To01(MultGenerator(C, i))

In [ ]:
m1 = 1
m2 = 3
m3 = 7
s = 3
a = 2

In [ ]:
m4 = (2**s * m1) ^ (m1) ^ (1 * 2**2 * m2) ^ (0 * 2**1 * m3)
m4

In [ ]:
m5 = (2**s * m2) ^ (m2) ^ (1 * 2**2 * m3) ^ (0 * 2**1 * m4)
m5

In [ ]:
#C2 = MakeTheMatrix(1,0,[1]) # Dim 2
#C3 = MakeTheMatrix(2,1,[1,3]) # Dim 3
#print(C9)
#print(C2)
TEST = MakeTheMatrix(3, 2, [1, 3, 7])
sorted([*Generate(16, TEST)])

In [ ]:
PlotMatrix(pyplot.gca(), TEST)
pyplot.show()
for v in TEST:
    print(bin(v))

In [ ]:
# https://web.maths.unsw.edu.au/~fkuo/sobol/
# d       s       a       m_i
joe_kue_D5_excerpt = r'''
2       1       0       1 
3       2       1       1 3 
4       3       1       1 3 1 
5       3       2       1 1 1 
6       4       1       1 1 3 3 
7       4       4       1 3 5 13 
8       5       2       1 1 5 5 17 
9       5       4       1 1 5 5 5 
10      5       7       1 1 7 11 19 
11      5       11      1 1 5 1 1 
12      5       13      1 3 7 1 19 
13      5       14      1 3 3 5 7 
14      6       1       1 3 3 13 9 53 
15      6       13      1 1 5 11 1 1 
16      6       16      1 1 3 7 21 51 
17      6       19      1 1 1 15 1 5 
18      6       22      1 3 1 9 9 1 
19      6       25      1 1 5 5 17 61 
20      7       1       1 3 1 15 29 57 87 
21      7       4       1 3 5 15 3 11 17 
22      7       7       1 3 3 7 5 17 65 
23      7       8       1 3 5 1 25 29 49 
24      7       14      1 1 3 7 15 39 119 
25      7       19      1 3 3 5 19 51 61 
26      7       21      1 1 5 15 11 47 15 
27      7       28      1 1 7 3 29 51 51 
28      7       31      1 1 3 15 19 17 13 
29      7       32      1 3 7 3 17 9 93 
30      7       37      1 3 7 5 7 29 111 
31      7       41      1 1 7 9 25 19 105 
32      7       42      1 1 1 11 21 35 107 
33      7       50      1 1 5 11 19 53 25 
34      7       55      1 3 1 3 27 29 31 
35      7       56      1 1 5 13 27 19 61 
36      7       59      1 3 1 3 25 33 105 
37      7       62      1 3 7 11 27 55 1 
38      8       14      1 1 7 1 9 45 97 63 
39      8       21      1 1 7 9 3 17 85 213 
40      8       22      1 1 1 3 31 35 93 35 
41      8       38      1 3 5 9 1 63 117 35 
42      8       47      1 3 1 9 21 3 53 29 
43      8       49      1 3 1 9 29 33 43 181 
44      8       50      1 3 7 3 21 45 121 141 
45      8       52      1 1 1 13 5 49 45 77 
46      8       56      1 1 3 3 1 47 37 151 
47      8       67      1 3 7 5 9 51 61 95 
48      8       70      1 1 1 7 31 23 81 105 
49      8       84      1 3 5 15 15 9 115 55 
50      8       97      1 3 3 13 15 1 87 11 
51      8       103     1 3 5 1 5 9 29 241 
52      8       115     1 1 1 9 19 5 115 191 
53      8       122     1 1 1 15 1 57 107 49 
54      9       8       1 1 7 7 23 21 71 187 207 
55      9       13      1 3 3 5 11 35 101 7 501 
56      9       16      1 3 5 15 29 5 61 205 301 
57      9       22      1 1 7 13 7 39 127 243 307 
58      9       25      1 3 7 13 29 9 93 187 429 
59      9       44      1 3 3 11 15 35 85 159 223 
60      9       47      1 1 3 1 13 3 111 17 411 
61      9       52      1 1 1 7 31 21 103 175 97 
62      9       55      1 1 1 15 11 21 63 45 29 
63      9       59      1 3 5 3 13 45 53 191 455 
64      9       62      1 3 3 13 11 37 65 45 371 
'''
def readln(row):
    data = [int(s.strip()) for s in row.split(' ') if s.strip()]
    d, s, a = data[:3]
    ms = data[3:]
    return d, s, a, ms

#generator_specs = [readln(r) for r in joe_kue_D5_excerpt.split('\n') if r]

with open("new-joe-kuo-5.21201","r") as f:
    joe_kue_D5_full = f.readlines()[1:]
generator_specs = [readln(r) for r in joe_kue_D5_full ]

In [ ]:
N = 100
def MakeGenerator(D):
    if D==1:
        return MakeIdentityGenerator()
    else:
        return MakeTheMatrix(*generator_specs[D-2][1:])
    
def Plot2d(N, D1, D2):
    col = np.arange(N)
    x1 = np.fromiter(Generate(N,MakeGenerator(D1)), dtype=np.float32)
    x2 = np.fromiter(Generate(N,MakeGenerator(D2)), dtype=np.float32)
    pyplot.scatter(x1,x2,c=col, marker = 'x')

In [ ]:
Plot2d(1000, 1, 2)

In [ ]:
Plot2d(1000, 3, 4)

In [ ]:
Plot2d(1000, 5, 6)

In [ ]:
Plot2d(1000, 7, 8)

In [ ]:
Plot2d(1000, 9, 10)

In [ ]:
Plot2d(100, 66, 67)

In [ ]:
Plot2d(1024, 64, 65)

In [ ]:
matrices = [ MakeIdentityGenerator() ]
for _, s, a, ms in generator_specs:
    Cd = MakeTheMatrix(s, a, ms)
    matrices.append(Cd)

In [ ]:
def CppMatrix(m):
    return '{'+','.join((hex(i)+'u') for i in m)+'}'
cpp = f'std::uint32_t sobol_generator_matrices[{len(matrices)}][32] = {{\n'+',\n'.join(CppMatrix(m) for m in matrices)+'\n};'
with open("../src/sampler_sobol_matrices.cxx", "w") as f:
    f.write("#include <cstdint>\n")
    f.write(cpp)

In [ ]:
print(fr"""
static constexpr int MAX_SOBOL_DIM = {len(matrices)};
extern std::uint32_t sobol_generator_matrices[MAX_SOBOL_DIM][32];
""")